In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
%matplotlib widget

### Reference: Influence of hole shape on sound absorption of underwater anechoic layers
#### https://www.sciencedirect.com/science/article/abs/pii/S0022460X1830227X

### Import pre-defined functions

In [3]:
%run -i elastic_modules.py

In [8]:
%run -i coefficient_matrix.py

In [10]:
determinant_01, mat = determinant_from_matrix()
mat

Matrix([
[                                                                                                              -sqrt(-kz**2 + omega**2/cl**2)*besselj(1, b*sqrt(-kz**2 + omega**2/cl**2)),                                                                                                               -sqrt(-kz**2 + omega**2/cl**2)*bessely(1, b*sqrt(-kz**2 + omega**2/cl**2)),                                                                                      -I*kz*sqrt(-kz**2 + omega**2/ct**2)*besselj(1, b*sqrt(-kz**2 + omega**2/ct**2)),                                                                                      -I*kz*sqrt(-kz**2 + omega**2/ct**2)*bessely(1, b*sqrt(-kz**2 + omega**2/ct**2))],
[(-2*mu*(-kz**2 + omega**2/cl**2) - lambda*omega**2/cl**2)*besselj(0, a*sqrt(-kz**2 + omega**2/cl**2)) + 2*mu*sqrt(-kz**2 + omega**2/cl**2)*besselj(1, a*sqrt(-kz**2 + omega**2/cl**2))/a, (-2*mu*(-kz**2 + omega**2/cl**2) - lambda*omega**2/cl**2)*bessely(0, a*sqrt(-kz**2 + omega**2/cl**

In [ ]:
# ai, lh_n = effective_radius(1,11, 40, 100, shape='horn')
# ai[-1]

In [ ]:
# plt.figure()
# plt.plot(lh_n, ai)

## Constants and Modulus

#### https://en.wikipedia.org/wiki/Lam%C3%A9_parameters
#### https://en.wikipedia.org/wiki/Shear_modulus

#### shear_modulus = Young_modulus/(2*(1+Poisson_ratio))
#### lame_const = Young_modulus*Poisson_ratio/((1+Poisson_ratio)*(1-2*Poisson_ratio))
#### longitudinal_modulus = lame_const + 2*shear_modulus
#### Young's modulus of rubber (GPa): 0.14 (incorrect) --> 14 MPa
#### Loss factor of rubber 0.23
#### Poisson ratio of rubber 0.49

All units are in SI

Density of rubber (kg/m3): 1100

Density of air (kg/m3): 1.21

Assume the pipe has a conical shape, as Fig. 7(a) in the paper:

pcone = 4 mm

qcone = 8 mm

lh = 40 mm

cell_radius = 15 mm

Assume 100 segments: num_segments = 100

Density of water (kg/m3): 998

Sound speed of water (m/s): 1483

In [ ]:
# def shear_m(Young_m, Poisson_r, loss_factor):
#     return Young_m*(1-1j*loss_factor)/(2*(1+Poisson_r))

# # def lame_const(Young_m, Poisson_r):
# #     return Young_m*Poisson_r/((1+Poisson_r)*(1-2*Poisson_r))

# def longitudinal_m(Yong_m, Poisson_r, loss_factor):
#     return Yong_m*(1-Poisson_r)*(1-1j*loss_factor)/((1+Poisson_r)*(1-2*Poisson_r))
    
# ## eq.(1-13b), in book page 26
# def ith_longitudinal_speed(longitudinal_m, density):
#     speed = np.sqrt(longitudinal_m/density)
#     # damping = np.sqrt(1+1j*loss_factor)
#     return speed    

# ## eq.(1-14), in book page 27
# def ith_transverse_speed(shear_m, density):
#     speed = np.sqrt(shear_m/density)
#     # damping = np.sqrt(1+1j*loss_factor)
#     return speed


### Test above defined functions for a single frequency

In [ ]:
frequency = 8000 #np.arange(0.1, 10000, 50)
omega = frequency * 2 * np.pi

Young_molulus = 14*(10**6)
Poisson_ratio = 0.49
loss_factor = 0.23
shear_modulus = shear_m(Young_molulus, Poisson_ratio, loss_factor)
# lame_constant = lame_const(Young_molulus, 0.49)
longitudinal_modulus = longitudinal_m(Young_molulus, Poisson_ratio, loss_factor)

ai, _ = effective_radius(4*0.001, 8*0.001, 40*0.001, 3, shape='cone')
effective_density = ith_effect_density(ai, 15*0.001, 1100, 1.21)

effective_speed = ith_longitudinal_speed(longitudinal_modulus, effective_density)

wave_number = ith_effect_wavenumber(omega, effective_speed)


effective_impedance = ith_effect_impedance(effective_density, omega, wave_number)
tn = total_tran_matrix(wave_number, 40*0.001, 3, effective_impedance)

## Acoustic impdedance of water: 1.48 MPa.s.m−1
## https://www.animations.physics.unsw.edu.au/jw/sound-impedance-intensity.htm

zw = 998*1483
zf = imped_front(tn)
ref = reflection_coefficient(zf, zw)
alpha = absorption_coefficient(ref)

In [ ]:
alpha

### Integrate all calculations into one function

In [ ]:
def absorption_frequency(frequency_array, p, q, lh, num_segments, 
                         Young_m, Poisson_r, loss_factor, 
                         cell_radius, shape='cone', rubber_den=1100, air_den=1.21):
    
    frequency_array = np.asarray(frequency_array)
    absorption_list = []
    
    for frequency in frequency_array:
       
        omega = frequency * 2 * np.pi

        ai, _ = effective_radius(p, q, lh, num_segments, shape=shape)
        effective_density = ith_effect_density(ai, cell_radius, rubber_den, air_den)
        
        longitudinal_modulus = longitudinal_m(Young_m, Poisson_r, loss_factor)
        # shear_modulus = shear_m(Young_m, Poisson_r, loss_factor)
        
        effective_speed = ith_longitudinal_speed(longitudinal_modulus, effective_density)
        wave_number = ith_effect_wavenumber(omega, effective_speed)
        
        effective_impedance = ith_effect_impedance(effective_density, omega, wave_number)
        tn = total_tran_matrix(wave_number, lh, num_segments, effective_impedance)

        ## Acoustic impdedance of water: 1.48 MPa.s.m−1
        ## https://www.animations.physics.unsw.edu.au/jw/sound-impedance-intensity.htm

        zw = 998*1483
        zf = imped_front(tn)
        ref = reflection_coefficient(zf, zw)
        alpha = absorption_coefficient(ref)   
        absorption_list.append(alpha)
        
    return np.asarray(absorption_list)


In [ ]:
frequency_array = np.arange(0.1, 10000, 100)
absorption_array = absorption_frequency(frequency_array, 4*0.001, 8*0.001, 40*0.001, 1000,
                                        0.014*(10**9), 0.49, 0.23, 
                                        15*0.001, shape='cone')

In [ ]:
df = pd.DataFrame()
df['KHZ'] = frequency_array*0.001
df['alpha-4-8mm'] = absorption_array

fn_path = '/Users/chenghunglin/Library/CloudStorage/OneDrive-BrookhavenNationalLaboratory/sound_absorption/'
fn = 'absorption_4-8mm_theoretical.csv'
# df.to_csv(fn_path+fn, header=True, sep=' ', float_format='{:.8e}'.format)

In [ ]:
rows = 1
cols = 1
f1, ax1 = plt.subplots(rows, cols, figsize = (6, 4), constrained_layout=True)
ax1.plot(frequency_array, absorption_array)
# plt.show()

In [ ]:
# plt.close('all')

## Plot more figures to compare with the paper

In [ ]:
## Set up constants
lh = 40 * 0.001
num_segments = 1000
Young_molulus = 14*(10**6)
Poisson_ratio = 0.49
loss_factor = 0.23
cell_radius = 15 * 0.001

In [ ]:
abs_cone_4_0 = absorption_frequency(frequency_array, 4*0.001, 0*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='cone')

In [ ]:
abs_cone_4_4 = absorption_frequency(frequency_array, 4*0.001, 4*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='cone')

In [ ]:
abs_cone_4_8 = absorption_frequency(frequency_array, 4*0.001, 8*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='cone')

In [ ]:
rows = 1
cols = 1
f2, ax2 = plt.subplots(rows, cols, figsize = (6, 4), constrained_layout=True)
ax2.plot(frequency_array, abs_cone_4_0, label='Cone, p=4, q=0')
ax2.plot(frequency_array, abs_cone_4_4, label='Cone, p=4, q=4')
ax2.plot(frequency_array, abs_cone_4_8, label='Cone, p=4, q=8')
ax2.legend()

In [ ]:
abs_horn_1_7 = absorption_frequency(frequency_array, 1*0.001, 7*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='horn')

In [ ]:
abs_horn_1_9 = absorption_frequency(frequency_array, 1*0.001, 9*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='horn')

In [ ]:
abs_horn_1_11 = absorption_frequency(frequency_array, 1*0.001, 11*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='horn')

In [ ]:
rows = 1
cols = 1
f3, ax3 = plt.subplots(rows, cols, figsize = (6, 4), constrained_layout=True)
ax3.plot(frequency_array, abs_horn_1_7, label='Horn, p=1, q=7')
ax3.plot(frequency_array, abs_horn_1_9, label='Horn, p=1, q=9')
ax3.plot(frequency_array, abs_horn_1_11, label='Horn, p=1, q=11')
ax3.legend()

In [ ]:
abs_cone_0_3 = absorption_frequency(frequency_array, 0*0.001, 3*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='cone')

In [ ]:
abs_cone_3_3 = absorption_frequency(frequency_array, 3*0.001, 3*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='cone')

In [ ]:
abs_cone_6_3 = absorption_frequency(frequency_array, 6*0.001, 3*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='cone')

In [ ]:
rows = 1
cols = 1
f4, ax4 = plt.subplots(rows, cols, figsize = (6, 4), constrained_layout=True)
ax4.plot(frequency_array, abs_cone_0_3, label='Cone, p=0, q=3')
ax4.plot(frequency_array, abs_cone_3_3, label='Cone, p=3, q=3')
ax4.plot(frequency_array, abs_cone_6_3, label='Cone, p=6, q=3')
ax4.legend()

In [ ]:
abs_horn_01_9 = absorption_frequency(frequency_array, 0.1*0.001, 9*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='horn')

In [ ]:
abs_horn_1_9 = absorption_frequency(frequency_array, 1*0.001, 9*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='horn')

In [ ]:
abs_horn_4_9 = absorption_frequency(frequency_array, 4*0.001, 9*0.001, lh, num_segments,
                                    Young_molulus, Poisson_ratio, loss_factor, 
                                    cell_radius, shape='horn')

In [ ]:
rows = 1
cols = 1
f5, ax5 = plt.subplots(rows, cols, figsize = (6, 4), constrained_layout=True)
ax5.plot(frequency_array, abs_horn_01_9, label='Horn, p=0.1, q=9')
ax5.plot(frequency_array, abs_horn_1_9, label='Horn, p=1, q=9')
ax5.plot(frequency_array, abs_horn_4_9, label='Horn, p=4, q=9')
ax5.legend()